## Read in data, remove refseq entries, and create list of query genes

In [1]:
#read in michaels table
mic=read.csv("~/Downloads/dREhomologs_MR20230718.csv")
mic=as.data.frame(mic)
#remove accession from refseq that start with NP/XP
mic <- mic[!grepl("^NP_|^XP_", mic[["Accession"]]), ]
mic


#find possible genes
genes=as.list(unique(mic$gene))

,X,Unnamed..0,Description,Scientific.Name,Common.Name,Taxid,Max.Score,Total.Score,Query.Cover,E.value,Per..Ident,Acc..Len,Accession,gene,ToLdiv
,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
2,1,1,FXN [synthetic construct],synthetic construct,,32630,429,429,100,2e-151,99.52,210,AKI71332.1,FXN,Synthetic
3,2,2,FXN [synthetic construct],synthetic construct,,32630,427,427,100,6e-151,99.52,210,AKI71333.1,FXN,Synthetic
4,3,3,frataxin [Homo sapiens],Homo sapiens,human,9606,426,426,100,2e-150,99.05,210,AAA98508.1,FXN,Primates
19,18,18,"RecName: Full=Frataxin, mitochondrial; Short=Fxn; Contains: RecName: Full=Frataxin intermediate form; Contains: RecName: Full=Frataxin mature form; Contains: RecName: Full=Extramitochondrial frataxin; Flags: Precursor [Macaca fascicularis]",Macaca fascicularis,crab-eating macaque,9541,393,393,100,3e-137,91.90,210,Q8HXX9.2,FXN,Primates
20,19,19,frataxin [Macaca fascicularis],Macaca fascicularis,crab-eating macaque,9541,386,386,100,1e-134,91.43,210,BAC20589.1,FXN,Primates
26,25,25,Friedreich ataxia protein [Macaca mulatta],Macaca mulatta,Rhesus monkey,9544,359,359,100,6e-124,86.19,210,EHH24154.1,FXN,Primates
27,26,26,"Chain A, Frataxin, mitochondrial [Homo sapiens]",Homo sapiens,human,9606,347,347,80,6e-120,100.00,169,5KZ5_A,FXN,Primates
32,31,31,frataxin [Homo sapiens],Homo sapiens,human,9606,329,329,76,2e-112,100.00,171,AAA98509.1,FXN,Primates
40,39,39,Hypothetical predicted protein [Marmota monax],Marmota monax,woodchuck,9995,328,391,99,6e-110,80.09,325,VTJ72052.1,FXN,Rodents


## Create accession lists for each query gene

In [2]:
# Your list of possible matches
values_list <- genes  # replace with your actual values

# Initialize an empty list to hold the lists of matching values
matching_values_lists <- list()

# Loop over each value in values_list
for(value in values_list) {
  
  # Subset df to rows where column1 matches the current value
  matching_rows <- mic[mic[["gene"]] == value, ]
  
  # Extract the values in column2 for these rows
  matching_values <- matching_rows[["Accession"]]
  
  # Convert the column to a list
  matching_values_list <- as.list(matching_values)
  
  # Add the list to matching_values_lists
  matching_values_lists[[value]] <- matching_values_list
}

# Print the list of lists
#print(matching_values_lists)
#matching_values_lists

In [3]:
names(matching_values_lists)

[1] "FXN"       "GLS"       "RAPGEF2"   "PRDM12"    "PABPN1"    "DAB1"     
 [7] "DMPK"      "CNBP"      "ATXN3"     "NOTCH2NLC" "TBX1"      "BEAN1"    
[13] "ATXN1"     "HTT"       "RFC1"      "JPH3"      "PHOX2B"    "CACNA1A"  
[19] "ATXN10"    "C9orf72"   "DIP2B"     "VWA1"      "SAMD12"    "DMD"      
[25] "TCF4"      "HOXD13"    "LRP12"     "NOP56"     "MARCHF6"   "ATXN2"    
[31] "CSTB"      "FOXL2"     "HOXA13"    "ZIC2"      "PRNP"      "COMP"     
[37] "EIF4A3"    "TNRC6A"    "YEATS2"    "FMR1"      "ZIC3"      "AR"       
[43] "ATN1"      "TBP"       "PPP2R2B"   "XYLT1"     "SOX3"      "AFF2"     
[49] "RUNX2"     "ATXN7"     "NIPA1"     "CBL"       "STARD7"    "ARX"      
[55] "GIPC1"

## Pull aa seq files for each query gene and put them in individual folders

### from uniprot

In [ ]:
# Load the required packages
library(httr)
library(purrr)

# List of lists with UniProt IDs
list_of_lists=matching_values_lists


# Function to retrieve and save the protein sequence for a given UniProt ID
get_protein_sequence <- function(id, list_name) {
  
  # Define the URL for the UniProt API
  url <- paste0("https://www.uniprot.org/uniprot/", id, ".fasta")
  
  # Send a GET request to the UniProt API
  response <- GET(url)
  
  # If the request was successful, save the protein sequence to a file
  if(status_code(response) == 200) {
    
    # Get the protein sequence from the response
    protein_sequence <- content(response, "text")
    
    # Define the directory for the output files
    output_dir <- paste0(list_name, "_aa")
    
    # Create the directory if it doesn't exist
    if(!dir.exists(output_dir)) {
      dir.create(output_dir)
    }
    
    # Define the path to the output file
    output_file <- file.path(output_dir, paste0(id, ".fasta"))
    
    # Write the protein sequence to the output file
    writeLines(protein_sequence, output_file)
    
  } else {
    
    # If the request was not successful, print a message and skip to the next ID
    message("Failed to retrieve protein sequence for UniProt ID: ", id)
    
  }
}

# Loop over each list in the list of lists
for(list_name in names(list_of_lists)) {
  
  # Get the current list of UniProt IDs
  uniprot_ids <- list_of_lists[[list_name]]
  
  # Retrieve and save the protein sequences for these UniProt IDs
  walk(uniprot_ids, get_protein_sequence, list_name = list_name)
  
}


Failed to retrieve protein sequence for UniProt ID: AKI71332.1

Failed to retrieve protein sequence for UniProt ID: AKI71333.1

Failed to retrieve protein sequence for UniProt ID: AAA98508.1

No encoding supplied: defaulting to UTF-8.

Failed to retrieve protein sequence for UniProt ID: BAC20589.1

Failed to retrieve protein sequence for UniProt ID: EHH24154.1

Failed to retrieve protein sequence for UniProt ID: 5KZ5_A

Failed to retrieve protein sequence for UniProt ID: AAA98509.1

Failed to retrieve protein sequence for UniProt ID: VTJ72052.1

Failed to retrieve protein sequence for UniProt ID: MBZ3873747.1

Failed to retrieve protein sequence for UniProt ID: KAF3815965.1

Failed to retrieve protein sequence for UniProt ID: KAF6484566.1

Failed to retrieve protein sequence for UniProt ID: PNJ81430.1

Failed to retrieve protein sequence for UniProt ID: KAI5934409.1

Failed to retrieve protein sequence for UniProt ID: KAF4018605.1

Failed to retrieve protein sequence for UniProt ID: KA

Failed to retrieve protein sequence for UniProt ID: NXB74554.1

Failed to retrieve protein sequence for UniProt ID: KFV60051.1

Failed to retrieve protein sequence for UniProt ID: KAF1517396.1

Failed to retrieve protein sequence for UniProt ID: NXV89671.1

Failed to retrieve protein sequence for UniProt ID: NXE14513.1

Failed to retrieve protein sequence for UniProt ID: KFO03977.1

Failed to retrieve protein sequence for UniProt ID: NXA92039.1

Failed to retrieve protein sequence for UniProt ID: KFW76582.1

Failed to retrieve protein sequence for UniProt ID: NXJ20595.1

Failed to retrieve protein sequence for UniProt ID: NWV91753.1

Failed to retrieve protein sequence for UniProt ID: RMC19262.1

Failed to retrieve protein sequence for UniProt ID: NXB85559.1

Failed to retrieve protein sequence for UniProt ID: NXE21298.1

Failed to retrieve protein sequence for UniProt ID: NWT00106.1

Failed to retrieve protein sequence for UniProt ID: NWX60658.1

Failed to retrieve protein sequence fo

Failed to retrieve protein sequence for UniProt ID: NXF70120.1

Failed to retrieve protein sequence for UniProt ID: OWK55558.1

Failed to retrieve protein sequence for UniProt ID: NXF89503.1

Failed to retrieve protein sequence for UniProt ID: NXL66372.1

Failed to retrieve protein sequence for UniProt ID: KFQ23568.1

Failed to retrieve protein sequence for UniProt ID: NXI40537.1

Failed to retrieve protein sequence for UniProt ID: KFQ01491.1

Failed to retrieve protein sequence for UniProt ID: KFO83925.1

Failed to retrieve protein sequence for UniProt ID: NXY90762.1

Failed to retrieve protein sequence for UniProt ID: NXR46442.1

Failed to retrieve protein sequence for UniProt ID: NXM79000.1

Failed to retrieve protein sequence for UniProt ID: NXI51868.1

Failed to retrieve protein sequence for UniProt ID: NWR98190.1

Failed to retrieve protein sequence for UniProt ID: NXN14586.1

Failed to retrieve protein sequence for UniProt ID: NWX22037.1

Failed to retrieve protein sequence for 

Failed to retrieve protein sequence for UniProt ID: KAF0033321.1

Failed to retrieve protein sequence for UniProt ID: KAF3851598.1

Failed to retrieve protein sequence for UniProt ID: TNN67636.1

Failed to retrieve protein sequence for UniProt ID: KAI4821438.1

Failed to retrieve protein sequence for UniProt ID: KAJ7993726.1

Failed to retrieve protein sequence for UniProt ID: KAI4899495.1

Failed to retrieve protein sequence for UniProt ID: KAH1168816.1

Failed to retrieve protein sequence for UniProt ID: KAG8011601.1

Failed to retrieve protein sequence for UniProt ID: PWA16030.1

Failed to retrieve protein sequence for UniProt ID: CAH1797943.1

Failed to retrieve protein sequence for UniProt ID: CAH3138834.1

Failed to retrieve protein sequence for UniProt ID: CAH1255681.1

Failed to retrieve protein sequence for UniProt ID: CAH3169329.1

Failed to retrieve protein sequence for UniProt ID: CAH3022577.1

Failed to retrieve protein sequence for UniProt ID: KAJ7380233.1

Failed to retr

Failed to retrieve protein sequence for UniProt ID: KAI8121195.1

Failed to retrieve protein sequence for UniProt ID: CAI9726705.1

Failed to retrieve protein sequence for UniProt ID: KAH8351277.1

Failed to retrieve protein sequence for UniProt ID: KXJ67817.1

Failed to retrieve protein sequence for UniProt ID: VDP40404.1

Failed to retrieve protein sequence for UniProt ID: RWS21189.1

Failed to retrieve protein sequence for UniProt ID: KAF6035420.1

Failed to retrieve protein sequence for UniProt ID: CAH2105830.1

Failed to retrieve protein sequence for UniProt ID: SPP88543.1

Failed to retrieve protein sequence for UniProt ID: KAI4457524.1

Failed to retrieve protein sequence for UniProt ID: RVE51988.1

Failed to retrieve protein sequence for UniProt ID: KAH9400615.1

Failed to retrieve protein sequence for UniProt ID: KAI2798785.1

Failed to retrieve protein sequence for UniProt ID: CAH0559584.1

Failed to retrieve protein sequence for UniProt ID: KAH8287284.1

Failed to retrieve p

...

Failed to retrieve protein sequence for UniProt ID: TKS66359.1

Failed to retrieve protein sequence for UniProt ID: KAI7808356.1

Failed to retrieve protein sequence for UniProt ID: RXN31403.1

Failed to retrieve protein sequence for UniProt ID: CAI5656672.1

Failed to retrieve protein sequence for UniProt ID: QXO15203.1

Failed to retrieve protein sequence for UniProt ID: KAA0715530.1

Failed to retrieve protein sequence for UniProt ID: KAF4108809.1

Failed to retrieve protein sequence for UniProt ID: RXN09274.1

Failed to retrieve protein sequence for UniProt ID: KAI4805496.1

Failed to retrieve protein sequence for UniProt ID: RXN27964.1

Failed to retrieve protein sequence for UniProt ID: CAB1328002.1

Failed to retrieve protein sequence for UniProt ID: TWW80342.1

Failed to retrieve protein sequence for UniProt ID: KAA0712074.1

Failed to retrieve protein sequence for UniProt ID: KAF5892991.1

Failed to retrieve protein sequence for UniProt ID: TRZ01582.1

Failed to retrieve prote

Failed to retrieve protein sequence for UniProt ID: OWK16879.1

Failed to retrieve protein sequence for UniProt ID: TRY87292.1

Failed to retrieve protein sequence for UniProt ID: KAF7210214.1

Failed to retrieve protein sequence for UniProt ID: KAA0711984.1

Failed to retrieve protein sequence for UniProt ID: EHB06531.1

Failed to retrieve protein sequence for UniProt ID: KAG8452501.1

Failed to retrieve protein sequence for UniProt ID: CAG6021696.1

Failed to retrieve protein sequence for UniProt ID: KAG1941097.1

Failed to retrieve protein sequence for UniProt ID: KAI4586609.1

Failed to retrieve protein sequence for UniProt ID: EQB77785.1

Failed to retrieve protein sequence for UniProt ID: KAJ8384106.1

Failed to retrieve protein sequence for UniProt ID: KAJ3602670.1

Failed to retrieve protein sequence for UniProt ID: KAJ8264822.1

Failed to retrieve protein sequence for UniProt ID: TKS70687.1

Failed to retrieve protein sequence for UniProt ID: KAJ8371955.1

Failed to retrieve p

Failed to retrieve protein sequence for UniProt ID: KFQ06507.1

Failed to retrieve protein sequence for UniProt ID: MCI4378834.1

Failed to retrieve protein sequence for UniProt ID: KAG8129434.1

Failed to retrieve protein sequence for UniProt ID: KAI7791220.1

Failed to retrieve protein sequence for UniProt ID: KAE8580065.1

Failed to retrieve protein sequence for UniProt ID: KAI1237034.1

Failed to retrieve protein sequence for UniProt ID: KAG7519857.1

Failed to retrieve protein sequence for UniProt ID: NXL24992.1

Failed to retrieve protein sequence for UniProt ID: KAG8585691.1

Failed to retrieve protein sequence for UniProt ID: VFV33139.1

Failed to retrieve protein sequence for UniProt ID: KAF6339392.1

Failed to retrieve protein sequence for UniProt ID: CAG5979042.1

Failed to retrieve protein sequence for UniProt ID: KPP74862.1

Failed to retrieve protein sequence for UniProt ID: VFV33136.1

Failed to retrieve protein sequence for UniProt ID: KAA8579618.1

Failed to retrieve p

Failed to retrieve protein sequence for UniProt ID: KAF7386972.1

Failed to retrieve protein sequence for UniProt ID: KAH0946401.1

Failed to retrieve protein sequence for UniProt ID: KAI0207402.1

Failed to retrieve protein sequence for UniProt ID: CAH2050521.1

Failed to retrieve protein sequence for UniProt ID: KAJ8736220.1

Failed to retrieve protein sequence for UniProt ID: AHB50509.1

Failed to retrieve protein sequence for UniProt ID: KAJ0182460.1

Failed to retrieve protein sequence for UniProt ID: CAF4793867.1

Failed to retrieve protein sequence for UniProt ID: CAD6221920.1

Failed to retrieve protein sequence for UniProt ID: EGI57471.1

Failed to retrieve protein sequence for UniProt ID: CAH2050522.1

Failed to retrieve protein sequence for UniProt ID: EFN75602.1

Failed to retrieve protein sequence for UniProt ID: CAB3240827.1

Failed to retrieve protein sequence for UniProt ID: KAG7189349.1

Failed to retrieve protein sequence for UniProt ID: CAB3240829.1

Failed to retrie

Failed to retrieve protein sequence for UniProt ID: VDP12474.1

Failed to retrieve protein sequence for UniProt ID: KAH7725993.1

Failed to retrieve protein sequence for UniProt ID: KAE9555747.1

Failed to retrieve protein sequence for UniProt ID: KOX80357.1

Failed to retrieve protein sequence for UniProt ID: KAH7700457.1

Failed to retrieve protein sequence for UniProt ID: PAA75899.1

Failed to retrieve protein sequence for UniProt ID: KAF8362812.1

Failed to retrieve protein sequence for UniProt ID: CAD7273403.1

Failed to retrieve protein sequence for UniProt ID: CAD7247269.1

Failed to retrieve protein sequence for UniProt ID: CAH2007886.1

Failed to retrieve protein sequence for UniProt ID: CAI4231203.1

Failed to retrieve protein sequence for UniProt ID: CAG5125648.1

Failed to retrieve protein sequence for UniProt ID: CAI4231202.1

Failed to retrieve protein sequence for UniProt ID: KAF6020077.1

Failed to retrieve protein sequence for UniProt ID: KAJ6642392.1

Failed to retrie

### from ncbi

In [ ]:
# Load the required packages
library(httr)
library(purrr)

# List of lists with NCBI IDs
list_of_lists <- matching_values_lists

# Function to retrieve and save the protein sequence for a given NCBI ID
get_protein_sequence <- function(id, list_name) {
  
  # Define the URL for the EFetch utility
  url <- "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
  
  # Define the parameters for the request
  params <- list(db = "protein", id = id, rettype = "fasta", retmode = "text")
  
  # Send a GET request to the EFetch utility
  response <- GET(url, query = params)
  
  # If the request was successful, save the protein sequence to a file
  if(status_code(response) == 200) {
    
    # Get the protein sequence from the response
    protein_sequence <- content(response, "text")
    
    # Define the directory for the output files
    output_dir <- paste0(list_name, "_aa_ncbi")
    
    # Create the directory if it doesn't exist
    if(!dir.exists(output_dir)) {
      dir.create(output_dir)
    }
    
    # Define the path to the output file
    output_file <- file.path(output_dir, paste0(id, ".fasta"))
    
    # Write the protein sequence to the output file
    writeLines(protein_sequence, output_file)
    
  } else {
    
    # If the request was not successful, print a message and skip to the next ID
    message("Failed to retrieve protein sequence for NCBI ID: ", id)
    
  }
}

# Loop over each list in the list of lists
for(list_name in names(list_of_lists)) {
  
  # Get the current list of NCBI IDs
  ncbi_ids <- list_of_lists[[list_name]]
  
  # Retrieve and save the protein sequences for these NCBI IDs
  walk(ncbi_ids, get_protein_sequence, list_name = list_name)
  
}